In [ ]:
import os
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from utils import find_repo_root, calculate_vectorized_correlation

### Function Definitions

In [ ]:
def test_model(layer, ROI, sub, X_test, y_test, test_results):
    """
    reads in all models (for all ROIs & subjects) for a certain stage
    tests the models & saves the predicted brain activations
    :return: overview over result measures
    """
    root_dir = find_repo_root()
    model_dir = os.path.join(root_dir, layer, ROI, sub)
    model = load_model(model_dir)
    test_corr = model.evaluate(X_test, y_test)
    
    # ToDo: save the predicted fmris
    # pred_fmri = np.zeros_like(fmri_val)
    # pred_fmri_save_path = os.path.join(results_dir, ROI + '_val.npy')
    
    return test_results
    

### Calculations

In [ ]:
# specify layer manually: in ["stage_1", "stage_2", "stage_3", "stage_4", "stage_5", "final"]
layer = 'stage_4'

subs = ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]
ROIs = ["WB", "V1", "V2","V3", "V4", "LOC", "EBA", "FFA","STS", "PPA"]

# test results dataframe
test_results = pd.DataFrame()

for ROI in ROIs:
    for sub in subs:
        # read in model
        X_test =
        y_test =
        test_results = test_model(layer, ROI, sub, X_test, y_test, test_results)

# calculate aggregated scores

# save the dataframe